In [135]:
import pandas as pd
# data = pd.read_csv('bulk_trades.csv')


In [130]:
# data.to_dict()

{'Trade Date': {0: '2021-01-21', 1: '2021-05-14', 2: '2020-06-29'},
 'Instrument Code': {0: 'TSLA', 1: 'GOOGL', 2: 'AMZN'},
 'Market Code': {0: 'ASX', 1: 'NZX', 2: 'NASDAQ'},
 'Quantity': {0: 100, 1: 2000, 2: 300},
 'Price': {0: 201.8, 1: 140.0, 2: 177.0},
 'Transaction Type': {0: 'BUY', 1: 'BUY', 2: 'BUY'},
 'Exchange Rate (optional)': {0: nan, 1: nan, 2: 0.7088527604},
 'Brokerage (optional)': {0: nan, 1: nan, 2: 2.345},
 'Brokerage Currency (optional)': {0: nan, 1: nan, 2: 'USD'},
 'Comments (optional)': {0: 'A sample comment, which will be stored in the comments field for this transaction',
  1: nan,
  2: nan}}

In [141]:
initial_amt = 100000
data = {'Trade Date': {0: '2021-01-21', 1: '2021-05-14', 2: '2020-06-29'},
 'Instrument Code': {0: 'TSLA', 1: 'GOOGL', 2: 'AMZN'},
 'Market Code': {0: 'ASX', 1: 'NZX', 2: 'NASDAQ'},
 'Quantity': {0: 3, 1: 4, 2: 2},
 'Price': {0: 201.8, 1: 140.0, 2: 177.0},
 'Transaction Type': {0: 'BUY', 1: 'BUY', 2: 'BUY'}}

data = pd.DataFrame(data)

In [142]:
def valid_ticker(ticker):
    live_price = yf.download(ticker,start=pd.Timestamp.now().date()-timedelta(days = 30), end = pd.Timestamp.now().date())[[
                        'Adj Close']]
    if len(live_price)<5:
        return False
    return True
    
def get_closing_price(ticker,date):
    try:
        live_price = yf.download(ticker, start=date, end=date+timedelta(days=1))[[
                            'Adj Close']].iloc[0][0]
    except:
        for x in range(10):
            print(f"data not available on {date}, taking previous date")
            try:
                live_price = yf.download(ticker, start=date-timedelta(days=x), end=date-timedelta(days=x-1))[[
                            'Adj Close']].iloc[0][0]
                break
            except:
                continue
            
    return live_price

In [143]:
## input trade date, ticker, qty, price, transaction type
def verify_data(data):
    for _,d in data.iterrows():
        live_price = get_closing_price(d['Instrument Code'],date)
        print(live_price)
        if (d['Price'] >= live_price * 0.5 or  d['Price'] <= live_price * 1.5) and valid_ticker(d['Instrument Code']):
            return True
        else:
            return False
verify_data(data)

[*********************100%%**********************]  1 of 1 completed
/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1387093021.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  live_price = yf.download(ticker, start=date, end=date+timedelta(days=1))[[
[*********************100%%**********************]  1 of 1 completed

201.8800048828125


True

In [144]:
txn_data = data[['Trade Date', 'Instrument Code', 'Market Code', 'Quantity', 'Price',
       'Transaction Type']]
txn_data.columns = ['Trade_Date', 'Instrument_Code', 'Market_Code', 'Quantity', 'Price',
       'Transaction_Type']
txn_data

def cash_pos(txn_type, price, qty):
       if txn_type == 'BUY':
              return -price * qty
       else:
              return price * qty
txn_data['cash_txn'] = txn_data.apply(lambda x: cash_pos(x.Transaction_Type, x.Price, x.Quantity),axis = 1)
txn_data

,Trade_Date,Instrument_Code,Market_Code,Quantity,Price,Transaction_Type,cash_txn
0,2021-01-21,TSLA,ASX,3,201.8,BUY,-605.4
1,2021-05-14,GOOGL,NZX,4,140.0,BUY,-560.0
2,2020-06-29,AMZN,NASDAQ,2,177.0,BUY,-354.0


In [145]:
list(set(txn_data['Instrument_Code']))

['GOOGL', 'AMZN', 'TSLA']

In [146]:
prices = yf.download(list(set(txn_data['Instrument_Code'])), start=txn_data['Trade_Date'].min(),end=txn_data['Trade_Date'].max())['Adj Close']
prices

[*********************100%%**********************]  3 of 3 completed


Ticker,AMZN,GOOGL,TSLA
Date,,,
2020-06-29,134.018997,69.858498,67.290001
2020-06-30,137.940994,70.902496,71.987335
2020-07-01,143.934998,72.099998,74.641998
2020-07-02,144.514999,73.496498,80.577332
2020-07-06,152.852005,74.982498,91.438667
...,...,...,...
2021-05-07,164.580505,117.596497,224.123337
2021-05-10,159.524506,114.587502,209.679993
2021-05-11,161.195496,113.502998,205.733337


In [165]:
def get_actual_holdings(txn_data,initial_amt):
    holdings = {}
    holdings['date'] = []
    holdings['tickers'] = []
    holdings['qty'] = []
    txn_data['Trade_Date']= pd.to_datetime(txn_data['Trade_Date'])
    for date in pd.date_range(start=txn_data['Trade_Date'].min(), end=pd.Timestamp.now().date(), freq='M'):
        trades_until_date = txn_data[txn_data['Trade_Date'] <= date]
        for _, trade in trades_until_date.iterrows():
            instrument = trade['Instrument_Code']
            quantity = trade['Quantity']
            # transaction_type = trade['Transaction_Type']
            holdings['date'] += [date]
            holdings['tickers'] += [instrument]
            holdings['qty'] += [quantity]
    return holdings

        

In [166]:
actual_holdings_data = pd.DataFrame(get_actual_holdings(txn_data,10000))
actual_holdings_data

/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1057203061.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  for date in pd.date_range(start=txn_data['Trade_Date'].min(), end=pd.Timestamp.now().date(), freq='M'):


,date,tickers,qty
0,2020-06-30,AMZN,2
1,2020-07-31,AMZN,2
2,2020-08-31,AMZN,2
3,2020-09-30,AMZN,2
4,2020-10-31,AMZN,2
...,...,...,...
112,2024-01-31,GOOGL,4
113,2024-01-31,AMZN,2
114,2024-02-29,TSLA,3
115,2024-02-29,GOOGL,4


In [167]:
actual_holdings_data['value']= actual_holdings_data.apply(lambda x: get_closing_price(x.tickers,x.date) * x.qty, axis = 1)
actual_holdings_data

[*********************100%%**********************]  1 of 1 completed
/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1387093021.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  live_price = yf.download(ticker, start=date, end=date+timedelta(days=1))[[
[*********************100%%**********************]  1 of 1 completed
/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1387093021.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  live_price = yf.download(ticker, start=date, end=date+timedelta(days=1))[[
[*********************100%%**********************]  1 of 1 completed
/var/fold

data not available on 2020-10-31 00:00:00, taking previous date
data not available on 2020-10-31 00:00:00, taking previous date


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['TSLA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-01-31 00:00:00 -> 2021-02-01 00:00:00)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['TSLA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-01-31 00:00:00 -> 2021-02-01 00:00:00)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['TSLA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-01-30 00:00:00 -> 2021-01-31 00:00:00)')
[*********************100%%**********************]  1 of 1 completed
/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1387093021.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by posi

data not available on 2021-01-31 00:00:00, taking previous date
data not available on 2021-01-31 00:00:00, taking previous date
data not available on 2021-01-31 00:00:00, taking previous date
data not available on 2021-01-31 00:00:00, taking previous date


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AMZN']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-01-31 00:00:00 -> 2021-02-01 00:00:00)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AMZN']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-01-30 00:00:00 -> 2021-01-31 00:00:00)')
[*********************100%%**********************]  1 of 1 completed
/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1387093021.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  live_price = yf.download(ticker, start=date-timedelta(days=x), end=date-timedelta(days=x-1))[[
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['

...

data not available on 2023-12-31 00:00:00, taking previous date
data not available on 2023-12-31 00:00:00, taking previous date
data not available on 2023-12-31 00:00:00, taking previous date


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AMZN']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-12-30 00:00:00 -> 2023-12-31 00:00:00)')
[*********************100%%**********************]  1 of 1 completed
/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1387093021.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  live_price = yf.download(ticker, start=date-timedelta(days=x), end=date-timedelta(days=x-1))[[
[*********************100%%**********************]  1 of 1 completed
/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1387093021.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consisten

data not available on 2023-12-31 00:00:00, taking previous date


[*********************100%%**********************]  1 of 1 completed
/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1387093021.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  live_price = yf.download(ticker, start=date, end=date+timedelta(days=1))[[
[*********************100%%**********************]  1 of 1 completed
/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1387093021.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  live_price = yf.download(ticker, start=date, end=date+timedelta(days=1))[[
[*********************100%%**********************]  1 of 1 completed
/var/fold

,date,tickers,qty,value
0,2020-06-30,AMZN,2,275.881989
1,2020-07-31,AMZN,2,316.467987
2,2020-08-31,AMZN,2,345.096008
3,2020-09-30,AMZN,2,314.872986
4,2020-10-31,AMZN,2,303.614990
...,...,...,...,...
112,2024-01-31,GOOGL,4,560.400024
113,2024-01-31,AMZN,2,310.399994
114,2024-02-29,TSLA,3,605.640015
115,2024-02-29,GOOGL,4,553.840027


In [168]:
portfolio_data = pd.DataFrame(actual_holdings_data.groupby('date')['value'].sum())
portfolio_data

,value
date,
2020-06-30,275.881989
2020-07-31,316.467987
2020-08-31,345.096008
2020-09-30,314.872986
2020-10-31,303.614990
2020-11-30,316.803986
2020-12-31,325.692993
2021-01-31,1114.150024
2021-02-28,984.793015


In [25]:
from datetime import timedelta
import yfinance as yf
df = pd.DataFrame(data)

# Convert 'Trade Date' column to datetime
df['Trade Date'] = pd.to_datetime(df['Trade Date'])

# Create an empty DataFrame to store MTM
mtm_df = pd.DataFrame(columns=['Date', 'MTM'])
mtm_df.set_index('Date', inplace=True)
error = 0 
spoiled_list = []
cash = 0
# Iterate over each day
for date in pd.date_range(start=df['Trade Date'].min(), end=pd.Timestamp.now().date(), freq='M'):
    # Filter trades up to the current date
    trades_until_date = df[df['Trade Date'] <= date]
    # Calculate MTM for each trade
    total_mtm = 0
    for _, trade in trades_until_date.iterrows():
        instrument = trade['Instrument Code']
        quantity = trade['Quantity']
        transaction_type = trade['Transaction Type']
        try:
            # Fetch live price from Yahoo Finance
            live_price = yf.download(instrument, start=date, end=date+timedelta(days=1))[[
                    'Adj Close']].iloc[0][0]
        except:
            error = 1
            spoiled_list += [date]
            break
        
        # Adjust total MTM based on transaction type
        if transaction_type == 'BUY':
            total_mtm += live_price * quantity
        elif transaction_type == 'SELL':
            total_mtm -= live_price * quantity
    
    ## if trade date:
    # Append the total MTM for the current date to the DataFrame
    if error == 1:
        error = 0 
        continue
    mtm_df.loc[date] = total_mtm

# Display the result
print(mtm_df)

/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1663119843.py:15: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  for date in pd.date_range(start=df['Trade Date'].min(), end=pd.Timestamp.now().date(), freq='M'):
[*********************100%%**********************]  1 of 1 completed
/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1663119843.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  live_price = yf.download(instrument, start=date, end=date+timedelta(days=1))[[
[*********************100%%**********************]  1 of 1 completed
/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1663119843.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, inte

                      MTM
Date                     
2020-06-30   41382.298279
2020-07-31   47470.198059
2020-08-31   51764.401245
2020-09-30   47230.947876
2020-11-30   47520.597839
2020-12-31   48853.948975
2021-03-31   68675.531006
2021-04-30   75659.300232
2021-06-30  318438.066101
2021-08-31  365980.841064
2021-09-30  342476.924133
2021-11-30  374559.730530
2021-12-31  374945.109558
2022-01-31  346703.044128
2022-02-28  345197.248840
2022-03-31  362954.260254
2022-05-31  288862.185669
2022-06-30  272236.327362
2022-08-31  282031.999969
2022-09-30  251725.003052
2022-10-31  242506.004333
2022-11-30  250411.995697
2023-01-31  245940.991974
2023-02-28  228959.996796
2023-03-31  259193.007660
2023-05-31  302307.005310
2023-06-30  304684.992981
2023-07-31  332286.999512
2023-08-31  339550.993347
2023-10-31  308171.002197
2023-11-30  332894.996643
2024-01-31  345489.010620
2024-02-29  350136.012268



/var/folders/qs/04dry4xx32bdxvwlc_9d8tgm0000gn/T/ipykernel_59880/1663119843.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  live_price = yf.download(instrument, start=date, end=date+timedelta(days=1))[[


In [28]:
trades_until_date

,Trade Date,Instrument Code,Market Code,Quantity,Price,Transaction Type,Exchange Rate (optional),Brokerage (optional),Brokerage Currency (optional),Comments (optional)
0,2021-01-21,TSLA,ASX,100,700.07,BUY,NaN,NaN,NaN,"A sample comment, which will be stored in the ..."
1,2021-05-14,GOOGL,NZX,2000,2500.65,BUY,NaN,NaN,NaN,NaN
2,2020-06-29,AMZN,NASDAQ,300,3020.89,BUY,0.708853,2.345,USD,NaN


In [26]:
spoiled_list

[Timestamp('2020-10-31 00:00:00'),
 Timestamp('2021-01-31 00:00:00'),
 Timestamp('2021-02-28 00:00:00'),
 Timestamp('2021-05-31 00:00:00'),
 Timestamp('2021-07-31 00:00:00'),
 Timestamp('2021-10-31 00:00:00'),
 Timestamp('2022-04-30 00:00:00'),
 Timestamp('2022-07-31 00:00:00'),
 Timestamp('2022-12-31 00:00:00'),
 Timestamp('2023-04-30 00:00:00'),
 Timestamp('2023-09-30 00:00:00'),
 Timestamp('2023-12-31 00:00:00')]

In [27]:
mtm_df

,MTM
Date,
2020-06-30,41382.298279
2020-07-31,47470.198059
2020-08-31,51764.401245
2020-09-30,47230.947876
2020-11-30,47520.597839
2020-12-31,48853.948975
2021-03-31,68675.531006
2021-04-30,75659.300232
2021-06-30,318438.066101


In [67]:
import yfinance as yf
import pandas as pd

# Define function to download historical prices
def download_prices(assets, start_date, end_date):
    data = yf.download(assets, start=start_date, end=end_date)['Adj Close']
    return data

# Define function to perform rebalancing
def rebalance_portfolio(portfolio_value, current_prices, weights):
    total_value = sum(portfolio_value)
    target_values = [total_value * weight for weight in weights]
    shares_to_buy = [target / price for target, price in zip(target_values, current_prices)]
    return shares_to_buy

# Define function to calculate portfolio metrics
def calculate_metrics(portfolio_value, current_prices):
    total_value = sum(portfolio_value)
    current_value = sum([shares * price for shares, price in zip(portfolio_value, current_prices)])
    portfolio_return = (current_value - total_value) / total_value
    return {
        'portfolio_value': current_value,
        'portfolio_return': portfolio_return
    }

# Define assets, initial cash, weights, and benchmark
assets = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
initial_cash = 10000
weights = [0.25, 0.25, 0.25, 0.25]
benchmark = 'SPY'

# Download historical prices
start_date = '2021-01-01'
end_date = '2021-12-31'
prices = download_prices(assets + [benchmark], start_date, end_date)

# Assign initial investment and weight per asset
initial_investment_per_asset = [initial_cash * weight for weight in weights]
portfolio_value = initial_investment_per_asset.copy()

# Prepare initial shares per asset
current_prices = prices.iloc[0].values
initial_shares_per_asset = [initial / price for initial, price in zip(initial_investment_per_asset, current_prices)]

# Initialize time series DataFrame for portfolio and individual stocks
time_series = pd.DataFrame(index=prices.index)

# Perform rebalancing (monthly in this example)
for i, date in enumerate(prices.index):
    if i != 0 and date.month != prices.index[i-1].month:
        shares_to_buy = rebalance_portfolio(portfolio_value, prices.iloc[i], weights)
        portfolio_value = [shares * price for shares, price in zip(shares_to_buy, prices.iloc[i])]
    
    # # Update time series DataFrame
    # time_series.loc[date, 'Portfolio'] = sum(portfolio_value)
    for asset in assets:
        time_series.loc[date, asset] = initial_shares_per_asset[assets.index(asset)] * prices.loc[date, asset]

# Calculate portfolio metrics
current_prices = prices.iloc[-1].values
metrics = calculate_metrics(portfolio_value, current_prices)
time_series['Portfolio'] = time_series[time_series.columns].sum(axis=1)
# Display the results
print("Time Series of Overall Portfolio and Individual Stocks:")
print(time_series.head())
print("\nPortfolio Metrics:")
print(metrics)


[*********************100%%**********************]  5 of 5 completed


Time Series of Overall Portfolio and Individual Stocks:
                   AAPL         MSFT        GOOGL         AMZN     Portfolio
Date                                                                        
2021-01-04  2500.000000  3320.204877  2500.000000  1882.413957  10202.618834
2021-01-05  2530.909349  3323.407584  2520.160681  1901.246300  10275.723914
2021-01-06  2445.715205  3237.234256  2495.292762  1853.911705  10032.153927
2021-01-07  2529.170536  3329.356426  2569.823811  1867.959031  10296.309803
2021-01-08  2551.000455  3349.641432  2603.844987  1880.092385  10384.579260

Portfolio Metrics:
{'portfolio_value': 2058519.4396972656, 'portfolio_return': 204.85194396972656}


In [69]:
prices

Ticker,AAPL,AMZN,GOOGL,MSFT,SPY
Date,,,,,
2021-01-04,127.002106,159.331497,86.306503,211.605286,352.767120
2021-01-05,128.572327,160.925507,87.002502,211.809402,355.196777
2021-01-06,124.244392,156.919006,86.143997,206.317352,357.320312
2021-01-07,128.483994,158.108002,88.717003,212.188538,362.629242
2021-01-08,129.592972,159.134995,89.891502,213.481354,364.695343
...,...,...,...,...,...
2021-12-23,174.066513,171.068497,146.916504,328.062103,456.154266
2021-12-27,178.065643,169.669495,147.906494,335.668427,462.609833
2021-12-28,177.038712,170.660995,146.686996,334.492188,462.231781
